In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [8]:
# 2. Create a temporary view of the DataFrame.

df = df.createOrReplaceTempView('home_sales')


In [16]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
avg_4_bed = spark.sql("""
    SELECT YEAR(date) AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
    ORDER BY year
""")
avg_4_bed.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [17]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
avg_3_bed = spark.sql("""
  SELECT date_built, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY date_built
  ORDER BY date_built
""")
avg_3_bed.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [18]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_3_bed_2fl = spark.sql("""
  SELECT date_built, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY date_built
  ORDER BY date_built
""")
avg_3_bed_2fl.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [19]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
avg_view = spark.sql("""
  SELECT view, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY view
  HAVING AVG(price) >= 350000
  ORDER BY view DESC
""")
avg_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  99| 840349.0|
|  96| 719316.0|
|  95| 812657.0|
|  86| 553193.0|
|  84| 997872.0|
|  77| 697708.0|
|  74| 795986.0|
|  71| 589309.0|
|  70| 559782.0|
|  68| 574888.0|
|  65| 665755.0|
|  63| 556377.0|
|  61| 890532.0|
|  60| 812820.0|
|  59| 758574.0|
|  56| 803578.5|
|  54| 596768.0|
|  53| 767327.0|
|  52| 568309.0|
+----+---------+

--- 0.6832060813903809 seconds ---


In [25]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")



DataFrame[]

In [26]:
# 8. Check if the table is cached.
spark.catalog.isCached("home_sales")

True

In [27]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()
avg_view = spark.sql("""
  SELECT view, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY view
  HAVING AVG(price) >= 350000
  ORDER BY view DESC
""")
avg_view.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|  99| 840349.0|
|  96| 719316.0|
|  95| 812657.0|
|  86| 553193.0|
|  84| 997872.0|
|  77| 697708.0|
|  74| 795986.0|
|  71| 589309.0|
|  70| 559782.0|
|  68| 574888.0|
|  65| 665755.0|
|  63| 556377.0|
|  61| 890532.0|
|  60| 812820.0|
|  59| 758574.0|
|  56| 803578.5|
|  54| 596768.0|
|  53| 767327.0|
|  52| 568309.0|
+----+---------+

--- 0.9740903377532959 seconds ---


In [29]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [32]:
# 11. Read the parquet formatted data.
p_home_sales = spark.read.parquet("home_sales_partitioned")

In [33]:
# 12. Create a temporary table for the parquet data.
p_home_sales.createOrReplaceTempView("p_home_sales_temp")

In [34]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
avg_view = spark.sql("""
  SELECT view, ROUND(AVG(price), 2) AS avg_price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
  GROUP BY view
  HAVING AVG(price) >= 350000
  ORDER BY view DESC
""")
avg_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|  99| 840349.0|
|  96| 719316.0|
|  95| 812657.0|
|  86| 553193.0|
|  84| 997872.0|
|  77| 697708.0|
|  74| 795986.0|
|  71| 589309.0|
|  70| 559782.0|
|  68| 574888.0|
|  65| 665755.0|
|  63| 556377.0|
|  61| 890532.0|
|  60| 812820.0|
|  59| 758574.0|
|  56| 803578.5|
|  54| 596768.0|
|  53| 767327.0|
|  52| 568309.0|
+----+---------+

--- 0.7286381721496582 seconds ---


In [35]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [36]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False